In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

eli5 = load_dataset("eli5_category", split="train[:5000]", trust_remote_code=True)

In [2]:
eli5 = eli5.train_test_split(test_size=0.2)

In [ ]:
eli5["train"][0]

In [3]:
eli5 = eli5.flatten()

In [ ]:
eli5["train"][0]

In [4]:


tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=eli5["train"].column_names,
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1944 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_eli5

In [5]:
block_size = 128

In [6]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [7]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=1)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="tf")

In [13]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = create_optimizer(learning_rate=2e-5, weight_decay_rate=0.01)

ImportError: cannot import name 'create_optimizer' from 'transformers' (c:\Users\shafi\.conda\envs\genAIenv\lib\site-packages\transformers\__init__.py)

In [ ]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    lm_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    lm_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [ ]:
# # from transformers.keras_callbacks import

# callback = PushToHubCallback(
#     output_dir="my_awesome_eli5_clm-model",
#     # tokenizer=tokenizer,
# )

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)